In [2]:
# predefinitions
import pandas as pd
directory="altman z_all(sheet1-2).xlsx"
fail_condition = 1.8 #must be equal to or below this number
naValues=["#N/A N/A", "#N/A Review"]

In [3]:
def getCompanyHistory(filename, skipRows, naValues=None):
    """
    Generates a Data Frame from a table of altman z values of companies
    throughout the years
    
    :param filename: an Excel file (.xlsx) containing a table of companies and
                    their altmen z values according to dates
    :type filename: str
    :param skipRows: a list of indexes, indicating which rows to skip from the
                    top of the file
    :type skipRows: list of int >= 0
    :param naValues: values in the given table that are to be considered "NaN"
    :type naValues: scalar, str, list-like, or dict, default None
    
    :returns: A Data Frame containing altman z values of companies
    :rtype: pandas.DataFrame
    
    Example:
    >> getCompanyHistory("altman z_all(sheet1-2).xlsx","Sheet1", [0,1,2,4,5],
                             ["#N/A N/A", "#N/A Review"])
    """
    # extracting data
    cols = pd.read_excel(filename, encoding="latin-1", skiprows = skipRows,
                         na_values = naValues, sheet_name = "Sheet1",
                         header=None, nrows=1, index_col = 0).values[0]
    data = pd.read_excel(filename, encoding="latin-1",
                           skiprows = skipRows, na_values = naValues,
                           sheet_name = "Sheet1", index_col = 0)
    
    # modificatons on the extracted data
    data.columns = cols # rename columns
    data=data.dropna(axis=1,how="all") # drop columns with no values
    return data.loc[:,~data.columns.duplicated()] # drop duplicated columns

In [4]:
def weedSuccessful(dataFrame, failCond):
    """
    Marks and weeds out companies that are considered not to have "failed" in the past
    from a given data frame of companies and their altman z values throughout
    the years
    
    :param dataFrame: A Data Frame containing altman z values of companies
    :type dataFrame: pandas.DateFrame
    :param failcond: a company which has ever had an altman z value less than
                    or equal to this number will be considered to have "failed"
                    in the past
    :type failcond: float
    
    :returns: A Data Frame containing only the altman z values of companies that have
                "failed" in the past
    :rtype: pandas.DataFrame
    """
    # weeding out companies that don't fail
    data_lower = dataFrame.where(dataFrame<=failCond)
    data_lower = data_lower.dropna(axis=1,how="all")
    return data_lower

In [5]:
def getCompanyFailHistory(dataFrame, markedData):
    """
    Removes the columns from `dataFrame` that don't exist in `markedData`
    
    :param dataFrame: A Data Frame containing altman z values of companies
    :type dataFrame: pandas.DataFrame
    :param markedData: A Data Frame containing only the altman z values of companies that have
                        "failed" in the past
    :type markedData: pandas.DataFrame
    
    :returns: A Data Frame containing only the altman z values of companies that have
                "failed" in the past
    :rtype: pandas.DataFrame
    """
    columns_to_keep=list(markedData.columns)
    return dataFrame.filter(columns_to_keep,axis=1)

In [6]:
data_r = getCompanyHistory(directory, [0,1,2,4,5], naValues)
data_lower = weedSuccessful(data_r, fail_condition)
data = getCompanyFailHistory(data_r, data_lower)

In [7]:
data_masked = data_lower.notna().applymap(lambda x: 1 if x else 0)
data_masked_counted = data_masked.apply(lambda y: y * (y.groupby((y != y.shift()).cumsum()).cumcount() + 1))
display(data_masked_counted)

,BA UN Equity,CAT UN Equity,CVX UN Equity,DOW UN Equity,VZ UN Equity,1COV GY Equity,BAYN GY Equity,BMW GY Equity,CON GY Equity,DAI GY Equity,...,SSW SJ Equity,STP SJ Equity,SUI SJ Equity,TFG SJ Equity,TGO SJ Equity,TKG SJ Equity,TSG SJ Equity,TXT SJ Equity,VKE SJ Equity,WBO SJ Equity
2000-03-31,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2000-06-30,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2000-09-29,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2000-12-29,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2001-03-30,0,0,0,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-06-28,0,0,0,0,0,0,0,59,0,64,...,11,18,15,0,0,0,0,47,8,0
2019-09-30,0,0,0,0,0,0,0,60,0,65,...,12,19,16,0,1,0,1,48,9,0
2019-12-31,0,0,0,0,0,0,0,61,0,66,...,0,20,17,0,2,0,2,49,10,0
2020-03-31,1,0,0,1,0,0,0,62,0,67,...,0,21,18,0,3,1,3,50,11,0


In [8]:
#The longest these companies have stayed below altman-z-score of 1.8
longest_under = data_masked_counted.apply(lambda x: x.max()).sort_values(ascending = False).to_frame()
longest_under

,0
ASUR UR Equity,80
PNM UN Equity,79
WEC UN Equity,79
NI UN Equity,79
SO UN Equity,79
...,...
3139 JT Equity,1
3116 JT Equity,1
3079 JT Equity,1
3036 JT Equity,1


In [9]:
#prototype for seeing the length of the consecutive quarters below 1.8

#data_mc_indexed = data_masked_counted["SO UN Equity"].value_counts().sort_values().to_frame()

data_mc_indexed = data_masked_counted.apply(lambda x: x.value_counts())[1:]

data_mc_indexed


,BA UN Equity,CAT UN Equity,CVX UN Equity,DOW UN Equity,VZ UN Equity,1COV GY Equity,BAYN GY Equity,BMW GY Equity,CON GY Equity,DAI GY Equity,...,SSW SJ Equity,STP SJ Equity,SUI SJ Equity,TFG SJ Equity,TGO SJ Equity,TKG SJ Equity,TSG SJ Equity,TXT SJ Equity,VKE SJ Equity,WBO SJ Equity
1,1.0,3.0,1.0,2.0,7.0,1.0,1.0,1.0,2.0,1.0,...,3.0,1.0,10.0,1.0,1.0,6.0,3.0,1.0,3.0,12.0
2,1.0,2.0,NaN,NaN,3.0,1.0,NaN,1.0,1.0,1.0,...,3.0,1.0,10.0,1.0,1.0,6.0,3.0,1.0,3.0,12.0
3,NaN,2.0,NaN,NaN,2.0,NaN,NaN,1.0,1.0,1.0,...,3.0,1.0,2.0,NaN,1.0,2.0,1.0,1.0,3.0,NaN
4,NaN,2.0,NaN,NaN,2.0,NaN,NaN,1.0,1.0,1.0,...,3.0,1.0,2.0,NaN,1.0,2.0,1.0,1.0,3.0,NaN
5,NaN,2.0,NaN,NaN,2.0,NaN,NaN,1.0,1.0,1.0,...,2.0,1.0,2.0,NaN,NaN,NaN,NaN,1.0,2.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
#prototype: number of times the company first fell below 1.8
data_falls = len(data_masked_counted[data_masked_counted["BA UN Equity"] == 1])
data_falls

1

In [11]:
data_masked_counted["BA UN Equity"].sum()

3

In [12]:
#Index_label = data_mc_indexed.index #.tolist()

times_went_under = data_mc_indexed[0:1]

x = data_mc_indexed[0:1]["CAT UN Equity"].tolist()[0]
#test = data_mc_indexed.loc[data_mc_indexed.index == 1,["BA UN Equity"]] 

test2 = data_mc_indexed.groupby("CAT UN Equity")

display(test2.indices)
display(test2.indices[x][-1]+1)
display(times_went_under)
display(x)

{1.0: array([5, 6], dtype=int64),
 2.0: array([1, 2, 3, 4], dtype=int64),
 3.0: array([0], dtype=int64)}

1

,BA UN Equity,CAT UN Equity,CVX UN Equity,DOW UN Equity,VZ UN Equity,1COV GY Equity,BAYN GY Equity,BMW GY Equity,CON GY Equity,DAI GY Equity,...,SSW SJ Equity,STP SJ Equity,SUI SJ Equity,TFG SJ Equity,TGO SJ Equity,TKG SJ Equity,TSG SJ Equity,TXT SJ Equity,VKE SJ Equity,WBO SJ Equity
1,1.0,3.0,1.0,2.0,7.0,1.0,1.0,1.0,2.0,1.0,...,3.0,1.0,10.0,1.0,1.0,6.0,3.0,1.0,3.0,12.0


3.0

In [22]:
test2.indices

{1.0: array([5, 6], dtype=int64),
 2.0: array([1, 2, 3, 4], dtype=int64),
 3.0: array([0], dtype=int64)}

In [13]:

columns=data_mc_indexed.columns
for company in columns:
    x = data_mc_indexed[0:1][company].tolist()[0]
    

In [50]:
def loops(dictionary_of_arrays_groupby):
    list_of_times = []
    for item in dictionary_of_arrays_groupby.items():
        list_of_times.append(item[-1])
    return list_of_times
def the_maxes(list_of_lists):
    the_please_final_list = []
    for item in list_of_lists:
        the_please_final_list.append(item[-1]+1)
    return the_please_final_list

In [51]:
data_final = data_mc_indexed.apply(lambda x: the_maxes(loops(data_mc_indexed.groupby(x).indices)))

        
data_final

BA UN Equity               [2]
CAT UN Equity        [7, 5, 1]
CVX UN Equity              [1]
DOW UN Equity              [1]
VZ UN Equity     [25, 6, 2, 1]
                     ...      
TKG SJ Equity           [4, 2]
TSG SJ Equity           [4, 2]
TXT SJ Equity             [51]
VKE SJ Equity      [14, 12, 4]
WBO SJ Equity              [2]
Length: 3971, dtype: object

In [54]:
data_final.to_frame()

,0
BA UN Equity,[2]
CAT UN Equity,"[7, 5, 1]"
CVX UN Equity,[1]
DOW UN Equity,[1]
VZ UN Equity,"[25, 6, 2, 1]"
...,...
TKG SJ Equity,"[4, 2]"
TSG SJ Equity,"[4, 2]"
TXT SJ Equity,[51]
VKE SJ Equity,"[14, 12, 4]"
